# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Откроем таблицу и изучим общую информацию о данных


In [4]:
import pandas as pd

data = pd.read_csv('data.csv')

In [5]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

In [9]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

In [12]:
data['days_employed'] = data['days_employed'].abs()

In [13]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть.

In [15]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

In [16]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [17]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

In [19]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [20]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных


In [22]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

In [24]:
data['education'] = data['education'].str.lower()

In [25]:
data.duplicated().sum()

71

In [26]:
data = data.drop_duplicates()

### Категоризация данных

In [28]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [29]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [30]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [31]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [32]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследование данных

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Создадим функцию, которая на основании числа детей у клиентов (столбец children`) будет присваивать категории:
- нет детей (число детей равно 0);
- 1 ребенок (число детей 1 или 2);
- 2 ребенка (число детей от 3 и более);
- 3 и более

In [36]:
# создаем функцию для категоризации
def category_children(children_number):
    if children_number == 0:
        return 'нет детей'
    elif 1 == children_number :
        return '1 ребенок'
    elif children_number == 2:
        return '2 ребенка'
    return '3 и более'
        

In [37]:
data['children_category'] = data['children'].apply(category_children) 

In [38]:
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,children_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью,1 ребенок
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем,1 ребенок
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью,нет детей
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования,3 и более
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы,нет детей
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью,нет детей
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью,нет детей
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования,нет детей
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы,2 ребенка
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью,нет детей


In [39]:
def pivot_table_ident(row, column_name):
    # переменная row принимает таблицу
    #переменная column_name принимает название столбца таблицы
    try:
        table = data.pivot_table(index = column_name, values = 'debt', aggfunc = ['sum','count', 'mean'])\
        .sort_values(by = ('mean', 'debt'))
        # переименнуем значения столбцов
        table.columns = ['debt', 'total', 'share']
        # переведем значения столбца share в проценты
        table['share'] = table['share'].map('{:.2%}'.format) 
        return table
    except:
        return 'неверное имя столбца'

In [40]:
pivot_table_ident(data, 'children_category')

,debt,total,share
children_category,,,
нет детей,1063,14091,7.54%
3 и более,31,380,8.16%
1 ребенок,444,4808,9.23%
2 ребенка,194,2052,9.45%


**Вывод:** количество детей влияет на возврат кредита в срок. Клиенты, которые не имеют детей, чаще возвращают кредит в срок, чем клиенты с детьми. Клиенты с 1 и 2 детьми реже других категорий возвращают кредит в срок.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
data.pivot_table(index = 'family_status', values = 'debt', aggfunc = ['count', 'sum', 'mean'])\
    .sort_values(by = ('mean', 'debt'))

,count,sum,mean
,debt,debt,debt
family_status,,,
вдовец / вдова,951,63,0.066246
в разводе,1189,84,0.070648
женат / замужем,12261,927,0.075606
гражданский брак,4134,385,0.093130
Не женат / не замужем,2796,273,0.097639


**Вывод:** семейное положение влияет на возврат кредита в срок. Клиенты в категории вдовец/вдовы возвращают кредит в срок чаще, чем другие категории. Люди, состоящие в браке или в разводе, возвращают кредит вовремя реже. Самый низкий процент у категории не женат/не замужем.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
pivot_table_ident(data, 'total_income_category')

,debt,total,share
total_income_category,,,
D,21,349,6.02%
B,354,5014,7.06%
A,2,25,8.00%
C,1353,15921,8.50%
E,2,22,9.09%


**Вывод:** уровень дохода влияет на срок возврата кредита в срок. В большенстве случаев клиенты с уровнем дохода 'D' возвращают кредит в срок. Категории 'A' и 'E' имеют маленькую выборку, и это приводит к невозможности обнаружить статистически значимые различия или связи между переменными. Следовательно, клиенты в категории 'C' чаще не возвращают кредит в срок.  Если сравним только две самые многочисленные выборки 'B' и 'C', то можно выявить зависимость между уровнем долговой нагрузки и категорией дохода. Это может указывать на то, что в общем случае люди с более высоким уровнем дохода имеют меньшую долговую нагрузку. 

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [49]:
pivot_table_ident(data, 'purpose_category')

,debt,total,share
purpose_category,,,
операции с недвижимостью,780,10751,7.26%
проведение свадьбы,183,2313,7.91%
получение образования,369,3988,9.25%
операции с автомобилем,400,4279,9.35%


**Вывод:** клиенты, которые взяли кредит на недвижимотсть, возвращают кредит в срок значительно чаще других категорий. Проценты категорий 'операции с автомобилем' и 'получение образования' находятся примерно на одном уровни и кредитополучатели данных групп чаще становятся должниками.

### Шаг 4: общий вывод.


Перед началом исследования получили таблицу с данными. В ходе просмотра таблицы были выявлены пропущенные данные, дубликаты, неверные типы данных столбцов, неправильные значения данных. С помощью методов unique, mean и median определили аномальные значения в данных и удалили из. Пропущенные значения были заменены медианным значением этого столбца. Дубликаты были удалены с помощью метода drop_duplicates. Тип данных количественных переменных был заменен на числовой формат вместо текстового. Отрицательные величины преобразованы в положительные.

В ходе исследования надежности заемщиков было выявлено:
1. Количество детей влияет на возврат кредита в срок. Клиенты, которые не имеют детей, чаще возвращают кредит в срок, чем клиенты с детьми. Клиенты с 1 и 2 детьми реже других категорий возвращают кредит в срок.
2. Семейное положение влияет на возврат кредита в срок. Клиенты в категории вдовец/вдовы возвращают кредит в срок чаще, чем другие категории, a категории не женат/не замужем реже всего.
3. Уровень дохода влияет на срок возврата кредита в срок. В большинстве случаев клиенты с уровнем дохода 'D' возвращают кредит в срок, клиенты в категории 'C' чаще не возвращают кредит в срок.. Если сравним только две самые многочисленные выборки 'B' и 'C', то можно выявить зависимость между уровнем долговой нагрузки и категорией дохода. Это может указывать на то, что в общем случае люди с более высоким уровнем дохода имеют меньшую долговую нагрузку.
4. Клиенты, которые взяли кредит на недвижимость, возвращают кредит в срок значительно чаще других категорий.

Банку следует больше обращать внимание при оценке способности потенциального заёмщика вернуть кредит на клиентов, которые попадают в категории: по числу детей - '2 ребенка', по семейному положению - 'не женат/не замужем', по уровню дохода - 'С', цель кредита - 'операции с автомобилем'. Данных клиенты реже всего возвращают кредит в срок и имеют большую вероятность стать должниками. 